In [232]:
import numpy as np
import pandas as pd
import haversine
import math

In [2]:
fn_out = "../input/deped_school_location_with_enrolment_2017_2018_consolidated.csv"

In [3]:
# read input files
fnA = "../data/deped_school_location_with_enrolment_2017_2018_validationA.csv"
dfA = pd.read_csv(fnA)

/opt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [79]:
fnB1 = "../data/deped_school_location_with_enrolment_2017_2018_geocoded1_validationB.csv"
dfB1 = pd.read_csv(fnB1)

In [75]:
fnB2 = "../data/deped_school_location_with_enrolment_2017_2018_geocoded2_validationB.csv"
dfB2 = pd.read_csv(fnB2)

In [76]:
len(dfA), len(dfB1), len(dfB2)

(47358, 47358, 47358)

In [7]:
dfA.head(2)

,SCHOOL ID,SCHOOL NAME,ADDRESS,REGION,DIVISION,MUNICIPALITY,BARANGGAY,LATITUDE,LONGITUDE,MAPPED,...,Reg_Name,Pro_Code,Pro_Name,Mun_Code,Mun_Name,Bgy_Code,Bgy_Name,MATCH_BARANGAY,MATCH_MUNICIPALITY,MATCH_REGION
0,100160,Adams CES,"Pob. 1, Adams, Ilocos Norte",Region I,Ilocos Norte,ADAMS,ADAMS (POB.),18.462302,120.901845,1,...,REGION I (ILOCOS REGION),PH012800000,ILOCOS NORTE,PH012801000,ADAMS,PH012801001,ADAMS (POB.),1.0,1.0,1.0
1,100163,Bucarot PS,"Bucarot, Adams, Ilocos Norte",Region I,Ilocos Norte,ADAMS,ADAMS (POB.),18.462088,120.903605,1,...,REGION I (ILOCOS REGION),PH012800000,ILOCOS NORTE,PH012801000,ADAMS,PH012801001,ADAMS (POB.),1.0,1.0,1.0


In [9]:
dfA.columns

Index([u'SCHOOL ID', u'SCHOOL NAME', u'ADDRESS', u'REGION', u'DIVISION',
       u'MUNICIPALITY', u'BARANGGAY', u'LATITUDE', u'LONGITUDE', u'MAPPED',
       u'INVALID', u'Reg_Code', u'Reg_Name', u'Pro_Code', u'Pro_Name',
       u'Mun_Code', u'Mun_Name', u'Bgy_Code', u'Bgy_Name', u'MATCH_BARANGAY',
       u'MATCH_MUNICIPALITY', u'MATCH_REGION'],
      dtype='object')

In [150]:
dfA["MAPPED"].value_counts()

1    42214
0     5144
Name: MAPPED, dtype: int64

In [204]:
df_out = dfA[[u'SCHOOL ID', u'SCHOOL NAME', u'DIVISION', u'ADDRESS']]
for col in [u'SCHOOL NAME', u'DIVISION', u'ADDRESS']:
    df_out[col] = df_out[col].str.upper()
df_out["DEPED_BARANGAY"] = dfA["BARANGGAY"].str.upper()
df_out["DEPED_MUNICIPALITY"] = dfA["MUNICIPALITY"].str.upper()
df_out["DEPED_REGION"] = dfA["REGION"].str.upper()
# set DEPED_COORD
df_out["DEPED_COORD"] = dfA["MAPPED"]
df_out.loc[dfA["INVALID"]==1, "DEPED_COORD"] = 2 # mapped and invalid
df_out.loc[(dfA["MAPPED"]==1) & dfA["INVALID"].isnull(), "DEPED_COORD"] = 3 # mapped and fell outside PH shapefile

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [205]:
df_out["DEPED_COORD"].value_counts()

1    41712
0     5144
2      367
3      135
Name: DEPED_COORD, dtype: int64

In [206]:
# set DEPED lon, lat - clean & convert to float
df_out["DEPED_LONGITUDE"] = dfA["LONGITUDE"].astype(float)
df_out["DEPED_LATITUDE"] = dfA["LATITUDE"].str.replace('\xc2','').str.replace('\xa0','').astype(float)

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [207]:
dfB1.columns

Index([u'SCHOOL ID', u'SCHOOL NAME', u'ADDRESS', u'REGION', u'DIVISION',
       u'MUNICIPALITY', u'BARANGGAY', u'LATITUDE', u'LONGITUDE', u'MAPPED',
       u'INVALID', u'GMAP_MAPPED', u'GMAP_LAT1', u'GMAP_LON1', u'GMAP_LAT2',
       u'GMAP_LON2', u'GMAP_DELTA_KM', u'Reg_Code', u'Reg_Name', u'Pro_Code',
       u'Pro_Name', u'Mun_Code', u'Mun_Name', u'Bgy_Code', u'Bgy_Name',
       u'MATCH_BARANGAY', u'MATCH_MUNICIPALITY', u'MATCH_REGION', u'DELTA'],
      dtype='object')

In [208]:
dfB2.columns

Index([u'SCHOOL ID', u'SCHOOL NAME', u'ADDRESS', u'REGION', u'DIVISION',
       u'MUNICIPALITY', u'BARANGGAY', u'LATITUDE', u'LONGITUDE', u'MAPPED',
       u'INVALID', u'GMAP_MAPPED', u'GMAP_LAT1', u'GMAP_LON1', u'GMAP_LAT2',
       u'GMAP_LON2', u'GMAP_DELTA_KM', u'Reg_Code', u'Reg_Name', u'Pro_Code',
       u'Pro_Name', u'Mun_Code', u'Mun_Name', u'Bgy_Code', u'Bgy_Name',
       u'MATCH_BARANGAY', u'MATCH_MUNICIPALITY', u'MATCH_REGION'],
      dtype='object')

In [209]:
# rename columns
dfB2.columns = [u'SCHOOL ID', u'SCHOOL NAME', u'ADDRESS', u'REGION', u'DIVISION',
       u'MUNICIPALITY', u'BARANGGAY', u'LATITUDE', u'LONGITUDE', u'MAPPED',
       u'INVALID', u'GMAP_MAPPED', u'GMAP_LAT1', u'GMAP_LON1', u'GMAP_LAT2',
       u'GMAP_LON2', u'GMAP_DELTA_KM', u'Reg_Code', u'Reg_Name', u'Pro_Code',
       u'Pro_Name', u'Mun_Code', u'Mun_Name', u'Bgy_Code', u'Bgy_Name',
       u'MATCH_BARANGAY', u'MATCH_MUNICIPALITY', u'MATCH_REGION']

In [210]:
len(dfB1[dfB1["GMAP_MAPPED"]==1]), dfB2["GMAP_MAPPED"].sum()

(5930, 483.0)

In [211]:
len(dfB1[(dfB1["GMAP_MAPPED"]==1) & (dfB2["GMAP_MAPPED"]==1)]) # intersection between B1 & B2

11

In [212]:
# fill in GMAP fields from B2 if not in B1
dfB = dfB1.copy()
for col in [u'GMAP_LAT1', u'GMAP_LON1', u'GMAP_LAT2',
       u'GMAP_LON2', u'GMAP_DELTA_KM', u'Reg_Code', u'Reg_Name', u'Pro_Code',
       u'Pro_Name', u'Mun_Code', u'Mun_Name', u'Bgy_Code', u'Bgy_Name', u'MATCH_BARANGAY', u'MATCH_MUNICIPALITY',
       u'MATCH_REGION', "GMAP_MAPPED", "INVALID"]:
    dfB.loc[dfB1["GMAP_MAPPED"]!=1, col] = dfB2.loc[dfB1["GMAP_MAPPED"]!=1, col]

In [213]:
dfB["GMAP_MAPPED"].value_counts()

1.0    6402
Name: GMAP_MAPPED, dtype: int64

In [214]:
len(dfB[dfB["GMAP_LAT1"].notnull() | dfB["GMAP_LAT2"].notnull()])

6402

In [215]:
dfB["INVALID"].value_counts()

0.0    6051
1.0     351
Name: INVALID, dtype: int64

In [216]:
len(dfB[dfB["GMAP_DELTA_KM"]==0]), len(dfB[(dfB["GMAP_DELTA_KM"]!=0) & (dfB["GMAP_MAPPED"]==1)])

(1747, 4655)

In [217]:
len(dfB[dfB["GMAP_DELTA_KM"]==0]) + len(dfB[(dfB["GMAP_DELTA_KM"]!=0) & (dfB["GMAP_MAPPED"]==1)])

6402

In [218]:
len(dfB[(dfB["GMAP_DELTA_KM"]!=0) & (dfB["GMAP_MAPPED"]==1)])*1./(len(dfB[dfB["GMAP_DELTA_KM"]==0]) + len(dfB[(dfB["GMAP_DELTA_KM"]!=0) & (dfB["GMAP_MAPPED"]==1)]))

0.7271165260855983

In [219]:
df_out["GMAP_COORD"] = -1
df_out.loc[(dfB["GMAP_MAPPED"]!=1) | (dfB["GMAP_DELTA_KM"]==0), "GMAP_COORD"] = 0 # not successfully geocoded in Google Maps
df_out.loc[(df_out["GMAP_COORD"]==-1) & (dfB["INVALID"]==0), "GMAP_COORD"] = 1 # geocoded and validated
df_out.loc[(df_out["GMAP_COORD"]==-1) & (dfB["INVALID"]==1), "GMAP_COORD"] = 2 # geocoded and invalid

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [220]:
df_out["GMAP_COORD"].value_counts()

0    42703
1     4383
2      272
Name: GMAP_COORD, dtype: int64

In [221]:
df_out["GMAP_LONGITUDE"] = dfB["GMAP_LON1"]
df_out["GMAP_LATITUDE"] = dfB["GMAP_LAT1"]
df_out["GMAP_DELTA_KM"] = dfB["GMAP_DELTA_KM"]

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [222]:
# define SOURCE_COORD
df_out["SOURCE_COORD"] = ""
df_out.loc[df_out["DEPED_COORD"]==1, "SOURCE_COORD"] = "DEPED"
df_out.loc[(df_out["DEPED_COORD"]!=1) & (df_out["GMAP_COORD"]==1), "SOURCE_COORD"] = "GMAP"

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [223]:
df_out["SOURCE_COORD"].value_counts()

DEPED    41712
GMAP      3621
          2025
Name: SOURCE_COORD, dtype: int64

In [224]:
df_out["SOURCE_COORD"].value_counts(1)

DEPED    0.880780
GMAP     0.076460
         0.042759
Name: SOURCE_COORD, dtype: float64

In [225]:
# get LON, LAT for source coord
subs = ["LONGITUDE", "LATITUDE"]
for sub in subs:
    df_out[sub] = np.nan
    df_out.loc[df_out["SOURCE_COORD"]=="DEPED", sub] = df_out.loc[df_out["SOURCE_COORD"]=="DEPED", "DEPED_"+sub] 
    df_out.loc[df_out["SOURCE_COORD"]=="GMAP", sub] = df_out.loc[df_out["SOURCE_COORD"]=="GMAP", "GMAP_"+sub] 

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [226]:
# get PSGC etc. for source coord
subs = ["Bgy_Code", "Bgy_Name", "Mun_Code", "Mun_Name", "Pro_Code", "Pro_Name", "Reg_Code", "Reg_Name"]
subs2 = ["PSGC_BARANGAY", "BARANGAY", "PSGC_MUNICIPALITY", "MUNICIPALITY", "PSGC_PROVINCE", "PROVINCE", "PSGC_REGION", "REGION"]
for i in np.arange(len(subs)):
    sub, sub2 = subs[i], subs2[i]
    df_out.loc[df_out["SOURCE_COORD"]=="DEPED", sub2] = dfA.loc[df_out["SOURCE_COORD"]=="DEPED", sub]
    df_out.loc[df_out["SOURCE_COORD"]=="GMAP", sub2] = dfA.loc[df_out["SOURCE_COORD"]=="GMAP", sub]

In [241]:
# get DELTA_KM
df_out["DELTA_KM"] = np.nan
for i in np.arange(len(df_out)):
    j = df_out.index[i]
    if((df_out.loc[j, "DEPED_COORD"]!=0) & (df_out.loc[j, "GMAP_COORD"]!=0)):
        lon1, lat1 = df_out.loc[j, "DEPED_LONGITUDE"], df_out.loc[j, "DEPED_LATITUDE"] # DepEd coordinates
        lon2, lat2 = df_out.loc[j, "GMAP_LONGITUDE"], df_out.loc[j, "GMAP_LATITUDE"] # GMap coordinates
        coord1, coord2 = (lon1, lat1), (lon2, lat2)
        delta_km = haversine.Haversine(coord1, coord2).km
        df_out.loc[j, "DELTA_KM"] = delta_km

In [242]:
len(df_out[(df_out["DEPED_COORD"]!=0) & (df_out["GMAP_COORD"]!=0)])

1143

In [243]:
df_out["DELTA_KM"].describe()

count     851.000000
mean       59.998836
std       166.974907
min         0.000000
25%         0.084804
50%         2.580416
75%        14.703794
max      1199.601328
Name: DELTA_KM, dtype: float64

In [244]:
df_out.columns

Index([u'SCHOOL ID', u'SCHOOL NAME', u'DIVISION', u'ADDRESS',
       u'DEPED_BARANGAY', u'DEPED_MUNICIPALITY', u'DEPED_REGION',
       u'DEPED_COORD', u'DEPED_LONGITUDE', u'DEPED_LATITUDE', u'GMAP_COORD',
       u'GMAP_LONGITUDE', u'GMAP_LATITUDE', u'GMAP_DELTA_KM', u'SOURCE_COORD',
       u'LONGITUDE', u'LATITUDE', u'PSGC_BARANGAY', u'BARANGAY',
       u'PSGC_MUNICIPALITY', u'MUNICIPALITY', u'PSGC_PROVINCE', u'PROVINCE',
       u'PSGC_REGION', u'REGION', u'DELTA_KM'],
      dtype='object')

In [245]:
df_out.to_csv(fn_out, index=False)

In [246]:
df_both = df_out[(df_out["DEPED_COORD"]!=0) & (df_out["GMAP_COORD"]!=0)]
len(df_both)

1143

In [247]:
df_both["DEPED_COORD"].value_counts()

1    811
2    227
3    105
Name: DEPED_COORD, dtype: int64

In [248]:
df_both["GMAP_COORD"].value_counts()

1    1063
2      80
Name: GMAP_COORD, dtype: int64